In [1]:
import scipy.io as sio
import numpy as np
import pandas as pd

In [2]:
def import_file():       

    all_data = pd.DataFrame()

    path = "E:\\Cerin_big-data\\"
    files = ["Test-Jan-01-Jan-05.xlsx", "Test-Jul-01-Jul-05.xlsx"]
    #files = ["Test-InvalidValues.xlsx"]

    for f in files:
        full_path = path + f
        print(full_path)
        df = pd.read_excel(full_path)
        all_data = all_data.append(df,ignore_index=True)

    return all_data

In [3]:
def show_info(column, percentil, last_size , current_size):
    print("Column: {:>7} \t Percentile: {:>18} \t Before/Current/Removed: {:>8} / {:>8} / {:>5}"
        .format(column, percentil, last_size, current_size, last_size - current_size))

In [4]:
def filter_data(dataset):
    last_size = len(dataset)
    for column in list(dataset):
        dataset = dataset[pd.to_numeric(dataset[column], errors='coerce').notnull()]
        percentil = np.percentile(dataset[column],1, interpolation='lower')
        dataset = dataset[dataset[column] > percentil]
        
        current_size = len(dataset[column])
        show_info(column, percentil, last_size, current_size)        
        last_size = current_size
    return dataset	

In [5]:
data = import_file()

E:\Cerin_big-data\Test-Jan-01-Jan-05.xlsx
E:\Cerin_big-data\Test-Jul-01-Jul-05.xlsx


In [7]:
filtered = filter_data(data)

Column:    Data 	 Percentile: 2018-01-01T07:35:30.000000000 	 Before/Current/Removed:    12717 /    12589 /   128
Column: Phase A Current 	 Percentile:                  0 	 Before/Current/Removed:    12589 /    10970 /  1619
Column: Phase B Current 	 Percentile: 0.0138623015955091 	 Before/Current/Removed:    10970 /    10860 /   110
Column: Phase C Current 	 Percentile: 0.0309892389923334 	 Before/Current/Removed:    10860 /    10751 /   109
Column: Current Total 	 Percentile:                  0 	 Before/Current/Removed:    10751 /     9656 /  1095
Column: Power Factor 	 Percentile:  -99.9995651245117 	 Before/Current/Removed:     9656 /     7339 /  2317
Column: Frequency 	 Percentile:   59.9704551696777 	 Before/Current/Removed:     7339 /     7265 /    74
Column: Apparent Power 	 Percentile:   62.9547309875488 	 Before/Current/Removed:     7265 /     7192 /    73
Column: Active Power 	 Percentile:   94.9208297729492 	 Before/Current/Removed:     7192 /     7120 /    72
Column: React